### install dependencies

In [64]:
! pip install pandas scikit-learn scipy

    100% |████████████████████████████████| 16.7MB 2.6MB/s eta 0:00:01   27% |████████▊                       | 4.5MB 25.2MB/s eta 0:00:01    82% |██████████████████████████▍     | 13.7MB 64.7MB/s eta 0:00:01


In [219]:
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [220]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

### read csv

In [221]:
df = pd.read_csv('/Users/zen/Downloads/Hackathon team tasks - Salesforce reachout data.csv')

### cleanup column names

In [223]:
def snake_case(s):
    return re.sub(r'\s+','_',re.sub(r'[^\w]',' ', s)).lower()

In [224]:
df.columns = [snake_case(c) for c in df.columns]

In [225]:
df.head()

,account_name,lead_source,type,created_date,stage,close_date,billing_state_province,customer_size_band
0,eShares Inc,Organic,Cancelled Customer,3/14/2017,Closed Lost,11/30/2017,California,151 - 200
1,Complete Nutrition,Sponsored Email,Cancelled Customer,4/6/2016,Disqualified,4/6/2016,Nebraska,201 - 250
2,Smart Metals Recycling,ZoomInfo,Cancelled Customer,11/10/2017,Disqualified,1/12/2018,North Carolina,51 - 100
3,Business Systems - Test Account,Blog Subscriber,Existing Business,5/14/2018,Closed Won,5/18/2018,NaN,1000+
4,Business Systems - Test Account,Blog Subscriber,Existing Business,5/18/2018,Qualified,7/31/2020,NaN,1000+


### create datetime

In [153]:
df['created_date'] = pd.to_datetime(df['created_date'])

### drop duplicates (keep newest created_date)

In [154]:
df = df.sort_values(by=['created_date'], ascending=False).drop_duplicates(subset='account_name')

### filter to one city

In [155]:
df = df[df['billing_stateprovince'].isin(['New York', 'California'])]

In [156]:
df['billing_stateprovince'].value_counts()

California    2470
New York      1316
Name: billing_stateprovince, dtype: int64

### filter to relevante stage

In [157]:
df = df[df['stage'].isin(('Closed Lost','Disqualified','Closed Won'))]

In [158]:
df['stage'].value_counts()

Closed Lost     1915
Disqualified     902
Closed Won       442
Name: stage, dtype: int64

### make bands into floats

In [148]:
def band_to_int(b):
    if not b:
        return None
    if b.lower() == 'null':
        return None
    if b == '1000+':
        return 1000.0
    else:
        return float(b.split(' - ')[1])

In [159]:
df['customer_size_band'] = df['customer_size_band'].apply(band_to_int).astype(float)

In [160]:
df['customer_size_band'].value_counts()

50.0      835
100.0     808
150.0     467
500.0     389
1000.0    297
200.0     283
250.0     171
Name: customer_size_band, dtype: int64

### drop dates

In [161]:
df.drop(['created_date','close_date'], inplace=True, axis=1)

### set name to index

In [162]:
df.set_index('account_name', inplace=True)

### drop nulls

In [177]:
df.dropna(inplace=True)

### make binary classifier

In [178]:
features = pd.get_dummies(df.drop(labels=['stage'], axis=1))
label = (df['stage'] == 'Closed Won')

In [179]:
features.head()

,customer_size_band,lead_source_Advertisement,lead_source_Affiliate,lead_source_Blog Subscription,lead_source_CAHR,lead_source_Capterra,lead_source_Client (Additional Business),lead_source_Client Referral,lead_source_Cold Email,lead_source_Consultant,...,lead_source_TheMakeGood,lead_source_Webinar,lead_source_Word of mouth,lead_source_ZoomInfo,type_Cancelled Customer,type_Existing Business,type_New Business,type_Renewal,billing_stateprovince_California,billing_stateprovince_New York
account_name,,,,,,,,,,,,,,,,,,,,,
Samasource,500.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
"Psyonix, Inc.",150.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
Bivio Networks,50.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
Americord Registry,50.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
Matador Network,50.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


### split into train/test

In [180]:
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=.25, random_state=1)

In [198]:
def scale_features(x_train, x_test, scaler_class=StandardScaler):
    sc = scaler_class().fit(x_train)
    x_train_scaled = sc.transform(x_train)
    x_test_scaled = sc.transform(x_test)
    
    x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns)
    x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns)
    return x_train_scaled,  x_test_scaled, sc

In [196]:
x_train_scaled, x_test_scaled, scaler = scale_features(x_train, x_test)

In [197]:
x_train_scaled.head()

,customer_size_band,lead_source_Advertisement,lead_source_Affiliate,lead_source_Blog Subscription,lead_source_CAHR,lead_source_Capterra,lead_source_Client (Additional Business),lead_source_Client Referral,lead_source_Cold Email,lead_source_Consultant,...,lead_source_TheMakeGood,lead_source_Webinar,lead_source_Word of mouth,lead_source_ZoomInfo,type_Cancelled Customer,type_Existing Business,type_New Business,type_Renewal,billing_stateprovince_California,billing_stateprovince_New York
0,-0.504448,-0.040572,-0.141836,-0.040572,-0.020274,-0.064229,-0.185537,-0.070389,-0.028677,-0.04537,...,-0.028677,-0.040572,-0.028677,-0.130894,-0.020274,-0.320709,0.321488,0.0,-1.330059,1.330059
1,2.798630,-0.040572,-0.141836,-0.040572,-0.020274,-0.064229,-0.185537,-0.070389,-0.028677,-0.04537,...,-0.028677,-0.040572,-0.028677,-0.130894,-0.020274,-0.320709,0.321488,0.0,-1.330059,1.330059
2,0.963586,-0.040572,-0.141836,-0.040572,-0.020274,-0.064229,-0.185537,-0.070389,-0.028677,-0.04537,...,-0.028677,-0.040572,-0.028677,-0.130894,-0.020274,-0.320709,0.321488,0.0,0.751847,-0.751847
3,-0.504448,-0.040572,-0.141836,-0.040572,-0.020274,-0.064229,-0.185537,-0.070389,-0.028677,-0.04537,...,-0.028677,-0.040572,-0.028677,-0.130894,-0.020274,-0.320709,0.321488,0.0,-1.330059,1.330059
4,0.963586,-0.040572,-0.141836,-0.040572,-0.020274,-0.064229,-0.185537,-0.070389,-0.028677,-0.04537,...,-0.028677,-0.040572,-0.028677,-0.130894,-0.020274,-0.320709,0.321488,0.0,0.751847,-0.751847


In [199]:
rf_model = RandomForestRegressor()

In [200]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [201]:
model.score(x_test_scaled, y_test)

0.9236453201970444

In [202]:
lr_model = LogisticRegression()

In [203]:
lr_model.fit(x_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [217]:
y_predict = lr_model.predict_proba(x_test_scaled)

In [218]:
y_predict

array([[0.92862836, 0.07137164],
       [0.92439578, 0.07560422],
       [0.94404194, 0.05595806],
       ...,
       [0.91314884, 0.08685116],
       [0.9416896 , 0.0583104 ],
       [0.95437688, 0.04562312]])

In [204]:
lr_model.score(x_test_scaled, y_test)

0.9248768472906403

In [214]:
lr_model_coeff = pd.DataFrame(lr_model.coef_, columns=x_test_scaled.columns).T
lr_model_coeff.columns = ['coeff']
lr_model_coeff.sort_values('coeff', ascending=False).head()

,coeff
type_Existing Business,0.739441
lead_source_Partner,0.159234
lead_source_External Referral,0.155977
lead_source_Demo Namely,0.124842
lead_source_Outbound Linkedin,0.116797


### export csv

In [46]:
df.to_csv('./clean.csv')